# SETUP / IMPORTS / CREDENTIALS

In [ ]:
# ====================================
# SETUP AND IMPORTS
# Execute to import all requirements for notebook to work
# ====================================

import json
import numpy as np
import pandas as pd
import hashlib
import requests
import time
import csv
import io
from pandas import json_normalize
from google.colab import files

In [ ]:
# ====================================
# SDAPI CREDENTIALS
# Set your keys here
# ====================================

outletApiKey = ''
secretKey = ''
delay_seconds = 30

In [ ]:
# ====================================
# OAUTH TOKEN GENERATOR
# Post function to obtain Bearer Token
# ====================================

def requestHeaders():
    timestamp = int(round(time.time() * 1000))
    outletApiKey
    secretKey

    post_url = 'https://oauth.performgroup.com/oauth/token/{0}?_fmt=json&_rt=b'.format(outletApiKey)

    # generate a unique hash
    key = str.encode(outletApiKey + str(timestamp) + secretKey)
    unique_hash = hashlib.sha512(key).hexdigest()

    # call the OAuth API (post)
    oauthHeaders = {'Content-Type': 'application/x-www-form-urlencoded',
               'Authorization': 'Basic {0}'.format(unique_hash),
               'Timestamp': '{0}'.format(str(timestamp))
               }

    BODY = {'grant_type': 'client_credentials',
            'scope': 'b2b-feeds-auth'}

    response = requests.post(post_url, data=BODY, headers=oauthHeaders)
    access_token = response.json()['access_token']
    oauthHeaders = {'Authorization': 'Bearer {0}'.format(access_token)}
    return oauthHeaders

# INDIVIDUAL BLOCKS

In [ ]:
# ====================================
# STEP 1: GET MA1 - Match IDs fetcher and DF storage for Chivas
# Function to  make GET request to MA1 - Fixtures by Chivas contestand ID: e603sojy77s4u0ypqds2v2a1g
# ====================================

def get_match_ids():
  # Initialize an empty list to store match IDs
  all_matches = []

  # API Parameters
  requestParameters = {
      "_fmt": "json",
      "_pgSz": "1",
      "_pgNm": "1",
      "live": "yes",
      "status": "played",
      "cvlv": "13,15",
      "ctst": "e603sojy77s4u0ypqds2v2a1g",
      "_rt" : "b"
  }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/match/{0}/?'.format(outletApiKey,)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)

  # Check response Status
  if response.status_code == 200:
      data = response.json()
  else:
      print(f"Error for tmcl {tmcl_id}: API request failed with status code {response.status_code}")
      print(response.text)

  # Extract matches ID
  matches = data.get('match', [])
  for match in matches:
      match_info = match.get('matchInfo', {})
      match_id = match_info.get('id', None)
      if match_id:
          all_matches.append({'Match ID': match_id})

  df_match_id = pd.DataFrame(all_matches)
  return df_match_id

matchId_df = get_match_ids()
matchId_df

,Match ID
0,f3523cj8oqwa3eg5pspyygx78


In [ ]:
# ====================================
# STEP 2: GET MA2 (Player Match Stats) - Player Match Stats fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA2 - Player Match Stats
# ====================================

def process_matchPlayerStats_data(match_id):

  # API Parameters
  requestParameters = {
      "_fmt" : "json",
      "detailed" : "yes",
      "fx" : match_id,
      "_rt" : "b",
      # "people" : "",
      # "_dlt" : "",
      # live" : "",
      # "status" : "",
      # "_pgNm" : "",
      # "_pgSz" : "",
      # "_fld" : "",
      # "_lcl" : "",
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchstats/{0}/?'.format(outletApiKey,)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)

  # Check response Status
  if response.status_code == 200:
    data = response.json()
  else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)  # Print the API response for debugging

  # Extract data
  match_info = data.get('matchInfo', {})
  competition_info = match_info.get('competition', {})
  stage_info = match_info.get('stage', {})
  live_data = data.get('liveData', {})
  line_ups = live_data.get('lineUp', [])

  # Create a DataFrame for xG player stats
  player_stats_data = []

  # Extract player stats
  for line_up in line_ups:
    for player in line_up.get('player', []):
      player_entry = {
          'Match ID':match_info.get('id','N/A'),
          'Competition ID': competition_info.get('id', 'N/A'),
          'Competition Name': competition_info.get('name', 'N/A'),
          'Week': match_info.get('week', 'N/A'),
          'Stage ID': stage_info.get('id', 'N/A'),
          'Stage Name': stage_info.get('name', 'N/A'),
          'Player ID': player.get('playerId', 'N/A'),
          'First Name': player.get('firstName', 'N/A'),
          'Last Name': player.get('lastName', 'N/A'),
          'Match Name': player.get('matchName', 'N/A'),
          'Shirt Number': player.get('shirtNumber', 'N/A'),
          'Position': player.get('position', 'N/A'),
          'Position Side': player.get('positionSide', 'N/A'),
          'Formation Place': player.get('formationPlace', 'N/A'),
      }

      for stat in player.get('stat', []):
        stat_type = stat.get('type', '')
        stat_value = stat.get('value', 0)
        player_entry[stat_type] = stat_value

      player_stats_data.append(player_entry)

  # Create a DataFrame and converts "NAN" to 0 value
  df_player_stats = pd.DataFrame(player_stats_data)
  df_player_stats = pd.DataFrame(player_stats_data).fillna(0)

  # Convert all stat columns (excluding metadata) to numeric
  non_stat_cols = ['Match ID', 'Competition ID', 'Competition Name','Week', 'Stage ID','Stage Name','Player ID', 'First Name', 'Last Name', 'Match Name',
                 'Shirt Number', 'Position', 'Position Side', 'Formation Place']
  stat_cols = [col for col in df_player_stats.columns if col not in non_stat_cols]

  for col in stat_cols:
    df_player_stats[col] = pd.to_numeric(df_player_stats[col], errors='coerce').fillna(0)

  # Optionally, convert all floats to ints if appropriate
  df_player_stats[stat_cols] = df_player_stats[stat_cols].astype(float)
  return df_player_stats

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_matchPlayerStats_data(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_playerStats_df = pd.concat(all_data, ignore_index=True)
    print(final_playerStats_df)
else:
    print("No match data found.")

# Export to Excel
excel_filename =  'Aggregated Season Player Stats.xlsx'
final_playerStats_df.to_excel(excel_filename, index=False)

# Provide download links
files.download(excel_filename)
print(f"Data has been saved to {excel_filename}")

In [ ]:
# ====================================
# STEP 3: GET MA2 (Team Match Stats) - Team Match Stats fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA2 - Team Match Stats
# ====================================

def process_matchTeamStats_data(match_id):

  # API Parameters
  requestParameters = {
      "_fmt" : "json",
      "detailed" : "yes",
      "fx" : match_id,
      "_rt" : "b",
      # "people" : "",
      # "_dlt" : "",
      # live" : "",
      # "status" : "",
      # "_pgNm" : "",
      # "_pgSz" : "",
      # "_fld" : "",
      # "_lcl" : "",
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchstats/{0}/?'.format(outletApiKey,)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)
  # Check response Status
  if response.status_code == 200:
    data = response.json()
  else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)  # Print the API response for debugging

  # Extract data
  match_info = data.get('matchInfo', {})
  competition_info = match_info.get('competition', {})
  stage_info = match_info.get('stage', {})
  live_data = data.get('liveData', {})
  line_ups = live_data.get('lineUp', [])

  # Create a DataFrame for team stats
  team_stats_data = []

  # Extract team stats
  for team_stats in line_ups:
      team_id = team_stats['contestantId']
      for stat in team_stats['stat']:
          stat_info = {
              'Match ID':match_info.get('id','N/A'),
              'Competition ID': competition_info.get('id', 'N/A'),
              'Competition Name': competition_info.get('name', 'N/A'),
              'Week': match_info.get('week', 'N/A'),
              'Stage ID': stage_info.get('id', 'N/A'),
              'Stage Name': stage_info.get('name', 'N/A'),
              'Team ID': team_id,
              'Stat Type': stat.get('type', 'N/A'),
              'Total': stat.get('value', 0)
          }
          team_stats_data.append(stat_info)

  # Create a DataFrame and converts "NAN" to 0 value
  df_team_stats = pd.DataFrame(team_stats_data)
  df_team_stats  = df_team_stats.pivot(index=['Team ID', 'Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID','Stage Name'], columns='Stat Type', values='Total').reset_index()
  df_team_stats = df_team_stats.fillna(0)

  # Convert all stat columns (excluding 'Team ID' and 'Match ID') to floats
  stat_cols = [col for col in df_team_stats.columns if col not in ['Team ID', 'Match ID', 'Competition ID', 'Competition Name','Week', 'Stage ID', 'Stage Name']]
  for col in stat_cols:
    df_team_stats[col] = pd.to_numeric(df_team_stats[col], errors='coerce').fillna(0).astype(float)

  return df_team_stats

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_matchTeamStats_data(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_teamStats_df = pd.concat(all_data, ignore_index=True)
    print(final_teamStats_df)
else:
    print("No match data found.")

# Export to Excel
excel_filename =  'Aggregated Season Team Stats.xlsx'
final_teamStats_df.to_excel(excel_filename, index=False)

# Provide download links
files.download(excel_filename)
print(f"Data has been saved to {excel_filename}")

In [ ]:
# ====================================
# STEP 4: GET MA12 (Player xG Stats) - Player xG Stats fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA12
# ====================================

def process_xG_matchPlayerStats_data(match_id):

  # API Parameters
  requestParameters = {
      "_fmt" : "json",
      "fx" : match_id,
      "_rt" : "b"
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchexpectedgoals/{0}/?'.format(outletApiKey)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)

  # Create a DataFrame for xG player stats
  player_xGstats_data = []

  # Check response Status
  if response.status_code == 200:
    data = response.json()

    # Extract data
    match_info = data.get('matchInfo', {})
    competition_info = match_info.get('competition', {})
    stage_info = match_info.get('stage', {})
    live_data = data.get('liveData', {})
    line_ups = live_data.get('lineUp', [])


    # Extract player stats
    for line_up in line_ups:
      for player in line_up.get('player', []):
        player_entry = {
            'Match ID':match_info.get('id','N/A'),
            'Competition ID': competition_info.get('id', 'N/A'),
            'Competition Name': competition_info.get('name', 'N/A'),
            'Week': match_info.get('week', 'N/A'),
            'Stage ID': stage_info.get('id', 'N/A'),
            'Stage Name': stage_info.get('name', 'N/A'),
            'Player ID': player.get('playerId', 'N/A'),
            'First Name': player.get('firstName', 'N/A'),
            'Last Name': player.get('lastName', 'N/A'),
            'Match Name': player.get('matchName', 'N/A'),
            'Shirt Number': player.get('shirtNumber', 'N/A'),
            'Position': player.get('position', 'N/A'),
            'Position Side': player.get('positionSide', 'N/A'),
            'Formation Place': player.get('formationPlace', 'N/A'),
        }

        for stat in player.get('stat', []):
          stat_type = stat.get('type', '')
          stat_value = stat.get('value', 0)
          player_entry[stat_type] = stat_value

        player_xGstats_data.append(player_entry)

  else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)  # Print the API response for debugging
    return None # Return None if the API call failed

  # Create a DataFrame and converts "NAN" to 0 value
  df_player_xGstats = pd.DataFrame(player_xGstats_data).fillna(0)

  # Convert all stat columns (excluding metadata) to numeric
  non_stat_cols = ['Match ID', 'Competiion ID', 'Competition Name','Week', 'Stage ID', 'Stage Name', 'Player ID', 'First Name', 'Last Name', 'Match Name',
                 'Shirt Number', 'Position', 'Position Side', 'Formation Place']
  stat_cols = [col for col in df_player_xGstats.columns if col not in non_stat_cols]

  for col in stat_cols:
    df_player_xGstats[col] = pd.to_numeric(df_player_xGstats[col], errors='coerce').fillna(0)

  # Optionally, convert all floats to ints if appropriate
  df_player_xGstats[stat_cols] = df_player_xGstats[stat_cols].astype(float)
  return df_player_xGstats

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_xG_matchPlayerStats_data(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_xG_playerStats_df = pd.concat(all_data, ignore_index=True)
    print(final_xG_playerStats_df)
else:
    print("No match data found.")

# Export to Excel
excel_filename =  'Aggregated xG Player Stats.xlsx'
final_xG_playerStats_df.to_excel(excel_filename, index=False)

# Provide download links
files.download(excel_filename)
print(f"Data has been saved to {excel_filename}")

In [ ]:
# ====================================
# STEP 5: GET MA12 (Player xG Stats) - Player xG Stats fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA12
# ====================================

def process_xG_matchTeamStats_data(match_id):

  # API Parameters
  requestParameters = {
      "_fmt" : "json",
      "fx" : match_id,
      "_rt" : "b"
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchexpectedgoals/{0}/?'.format(outletApiKey)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)

  # Check response Status
  if response.status_code == 200:
    data = response.json()
  else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)  # Print the API response for debugging

  # Extract data
  match_info = data.get('matchInfo', {})
  competition_info = match_info.get('competition', {})
  stage_info = match_info.get('stage', {})
  live_data = data.get('liveData', {})
  line_ups = live_data.get('lineUp', [])

  # Create a DataFrame for xG team stats
  team_xGstats_data = []

  # Extract team stats
  for team_stats in line_ups:
      team_id = team_stats['contestantId']
      for stat in team_stats['stat']:
          stat_info = {
              'Match ID':match_info.get('id','N/A'),
              'Competition ID': competition_info.get('id', 'N/A'),
              'Competition Name': competition_info.get('name', 'N/A'),
              'Week': match_info.get('week', 'N/A'),
              'Stage ID': stage_info.get('id', 'N/A'),
              'Stage Name': stage_info.get('name', 'N/A'),
              'Team ID': team_id,
              'Stat Type': stat.get('type', 'N/A'),
              'Total': stat.get('value', 0)
          }
          team_xGstats_data.append(stat_info)

  # Create a DataFrame and converts "NAN" to 0 value
  df_xGteam_stats = pd.DataFrame(team_xGstats_data)
  df_xGteam_stats  = df_xGteam_stats.pivot(index=['Team ID', 'Match ID', 'Competition ID', 'Competition Name','Week', 'Stage ID', 'Stage Name'], columns='Stat Type', values='Total').reset_index()
  df_xGteam_stats = df_xGteam_stats.fillna(0)

  # Convert all stat columns (excluding 'Team ID' and 'Match ID') to floats
  stat_cols = [col for col in df_xGteam_stats.columns if col not in ['Team ID', 'Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name']]
  for col in stat_cols:
    df_xGteam_stats[col] = pd.to_numeric(df_xGteam_stats[col], errors='coerce').fillna(0).astype(float)

  return df_xGteam_stats

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_xG_matchTeamStats_data(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_xG_teamStats_df = pd.concat(all_data, ignore_index=True)
    print(final_xG_teamStats_df)
else:
    print("No match data found.")

# Export to Excel
excel_filename =  'Aggregated xG Team Stats.xlsx'
final_xG_teamStats_df.to_excel(excel_filename, index=False)

# Provide download links
files.download(excel_filename)
print(f"Data has been saved to {excel_filename}")

In [ ]:
# ====================================
# STEP 6: GET MA12 (Player xG Events) - Player xG Events fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA12
# ====================================

def process_xG_match_events(match_id):

  # API Parameters
  requestParameters = {
      "_rt" : "b",
      "_fmt" : "json",
      "fx" : match_id
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchexpectedgoals/{0}/?'.format(outletApiKey)

  response = requests.get(
      sdapi_get_url,
      headers = requestHeaders(),
      params=requestParameters
      )

  # Check response Status
  if response.status_code == 200:
      data = response.json()
  else:
      print(f"Error: API request failed with status code {response.status_code}")
      print(response.text)  # Print the API response for debugging

  # Extract event data
  match_info = data.get('matchInfo', {})
  competition_info = match_info.get('competition', {})
  stage_info = match_info.get('stage', {})
  live_data = data.get('liveData', {})
  xG_events = live_data.get('event', [])


  # Find all unique qualifier IDs
  qualifier_ids = {'321','322'}

  # Initialize DataFrame columns
  columns = [
      'Match ID', 'Competition ID', 'Competition Name','Week', 'Stage ID', 'Stage Name','EventId', 'timeStamp', 'contestantId', 'periodId', 'timeMin', 'timeSec',
      'playerId', 'playerName', 'typeId', 'outcome', 'x', 'y'] + [f'qualifier {qid}' for qid in qualifier_ids]

  # Create a DataFrame for xG events
  xG_events_data = []

  for event in xG_events:
    xG_event_info = {
        'Match ID':match_info.get('id','N/A'),
        'Competition ID': competition_info.get('id', 'N/A'),
        'Competition Name': competition_info.get('name', 'N/A'),
        'Week': match_info.get('week', 'N/A'),
        'Stage ID': stage_info.get('id', 'N/A'),
        'Stage Name': stage_info.get('name', 'N/A'),
        'EventId': event.get('eventId', None),
        'typeId': event.get('typeId', None),
        'periodId': event.get('periodId', None),
        'timeMin': event.get('timeMin', None),
        'timeSec': event.get('timeSec', None),
        'contestantId': event.get('contestantId', None),
        'playerId': event.get('playerId', None),
        'playerName': event.get('playerName', None),
        'outcome': event.get('outcome', None),
        'x': event.get('x', None),
        'y': event.get('y', None),
        'timeStamp': event.get('timeStamp', None),
        }

    for qid in qualifier_ids:
      xG_event_info[f'qualifier {qid}'] = 0

    # Update with actual qualifier values
    for q in event.get('qualifier', []):
      xG_event_info[f'qualifier {q["qualifierId"]}'] = q.get('value', None)

    xG_events_data.append(xG_event_info)

  xG_events_df = pd.DataFrame(xG_events_data, columns=columns)
  return xG_events_df

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_xG_match_events(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_xG_events_df = pd.concat(all_data, ignore_index=True)
    print(final_xG_teamStats_df)
else:
    print("No match data found.")

# Export to Excel
excel_filename =  'Aggregated xG events.xlsx'
final_xG_events_df.to_excel(excel_filename, index=False)

# Provide download links
files.download(excel_filename)
print(f"Data has been saved to {excel_filename}")

# MAIN PIPELINE IN ONE BLOCK

In [ ]:
# ====================================
# STEP 1: GET MA1 - Match IDs fetcher and DF storage for Chivas
# Function to  make GET request to MA1 - Fixtures by Chivas contestand ID: e603sojy77s4u0ypqds2v2a1g
# ====================================

def get_match_ids():
  # Initialize an empty list to store match IDs
  all_matches = []

  # API Parameters
  requestParameters = {
      "_fmt": "json",
      "_pgSz": "1",
      "_pgNm": "1",
      "live": "yes",
      "status": "played",
      "cvlv": "13,15",
      "ctst": "e603sojy77s4u0ypqds2v2a1g",
      "_rt" : "b"
  }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/match/{0}/?'.format(outletApiKey,)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)

  # Check response Status
  if response.status_code == 200:
      data = response.json()
  else:
      print(f"Error for tmcl {tmcl_id}: API request failed with status code {response.status_code}")
      print(response.text)

  # Extract matches ID
  matches = data.get('match', [])
  for match in matches:
      match_info = match.get('matchInfo', {})
      match_id = match_info.get('id', None)
      if match_id:
          all_matches.append({'Match ID': match_id})

  df_match_id = pd.DataFrame(all_matches)
  return df_match_id

matchId_df = get_match_ids()
matchId_df

# ====================================
# STEP 2: GET MA2 (Player Match Stats) - Player Match Stats fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA2 - Player Match Stats
# ====================================

def process_matchPlayerStats_data(match_id):

  # API Parameters
  requestParameters = {
      "_fmt" : "json",
      "detailed" : "yes",
      "fx" : match_id,
      "_rt" : "b",
      # "people" : "",
      # "_dlt" : "",
      # live" : "",
      # "status" : "",
      # "_pgNm" : "",
      # "_pgSz" : "",
      # "_fld" : "",
      # "_lcl" : "",
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchstats/{0}/?'.format(outletApiKey,)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)

  # Check response Status
  if response.status_code == 200:
    data = response.json()
  else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)  # Print the API response for debugging

  # Extract data
  match_info = data.get('matchInfo', {})
  competition_info = match_info.get('competition', {})
  stage_info = match_info.get('stage', {})
  live_data = data.get('liveData', {})
  line_ups = live_data.get('lineUp', [])

  # Create a DataFrame for xG player stats
  player_stats_data = []

  # Extract player stats
  for line_up in line_ups:
    for player in line_up.get('player', []):
      player_entry = {
          'Match ID':match_info.get('id','N/A'),
          'Competition ID': competition_info.get('id', 'N/A'),
          'Competition Name': competition_info.get('name', 'N/A'),
          'Week': match_info.get('week', 'N/A'),
          'Stage ID': stage_info.get('id', 'N/A'),
          'Stage Name': stage_info.get('name', 'N/A'),
          'Player ID': player.get('playerId', 'N/A'),
          'First Name': player.get('firstName', 'N/A'),
          'Last Name': player.get('lastName', 'N/A'),
          'Match Name': player.get('matchName', 'N/A'),
          'Shirt Number': player.get('shirtNumber', 'N/A'),
          'Position': player.get('position', 'N/A'),
          'Position Side': player.get('positionSide', 'N/A'),
          'Formation Place': player.get('formationPlace', 'N/A'),
      }

      for stat in player.get('stat', []):
        stat_type = stat.get('type', '')
        stat_value = stat.get('value', 0)
        player_entry[stat_type] = stat_value

      player_stats_data.append(player_entry)

  # Create a DataFrame and converts "NAN" to 0 value
  df_player_stats = pd.DataFrame(player_stats_data)
  df_player_stats = pd.DataFrame(player_stats_data).fillna(0)

  # Convert all stat columns (excluding metadata) to numeric
  non_stat_cols = ['Match ID', 'Competition ID', 'Competition Name','Week', 'Stage ID', 'Stage Name','Player ID', 'First Name', 'Last Name', 'Match Name',
                 'Shirt Number', 'Position', 'Position Side', 'Formation Place']
  stat_cols = [col for col in df_player_stats.columns if col not in non_stat_cols]

  for col in stat_cols:
    df_player_stats[col] = pd.to_numeric(df_player_stats[col], errors='coerce').fillna(0)

  # Optionally, convert all floats to ints if appropriate
  df_player_stats[stat_cols] = df_player_stats[stat_cols].astype(float)
  return df_player_stats

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_matchPlayerStats_data(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_playerStats_df = pd.concat(all_data, ignore_index=True)
    print(final_playerStats_df)
else:
    print("No match data found.")

# ====================================
# STEP 3: GET MA2 (Team Match Stats) - Team Match Stats fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA2 - Team Match Stats
# ====================================

def process_matchTeamStats_data(match_id):

  # API Parameters
  requestParameters = {
      "_fmt" : "json",
      "detailed" : "yes",
      "fx" : match_id,
      "_rt" : "b",
      # "people" : "",
      # "_dlt" : "",
      # live" : "",
      # "status" : "",
      # "_pgNm" : "",
      # "_pgSz" : "",
      # "_fld" : "",
      # "_lcl" : "",
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchstats/{0}/?'.format(outletApiKey,)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)
  # Check response Status
  if response.status_code == 200:
    data = response.json()
  else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)  # Print the API response for debugging

  # Extract data
  match_info = data.get('matchInfo', {})
  competition_info = match_info.get('competition', {})
  stage_info = match_info.get('stage', {})
  live_data = data.get('liveData', {})
  line_ups = live_data.get('lineUp', [])

  # Create a DataFrame for team stats
  team_stats_data = []

  # Extract team stats
  for team_stats in line_ups:
      team_id = team_stats['contestantId']
      for stat in team_stats['stat']:
          stat_info = {
              'Match ID':match_info.get('id','N/A'),
              'Competition ID': competition_info.get('id', 'N/A'),
              'Competition Name': competition_info.get('name', 'N/A'),
              'Week': match_info.get('week', 'N/A'),
              'Stage ID': stage_info.get('id', 'N/A'),
              'Stage Name': stage_info.get('name', 'N/A'),
              'Team ID': team_id,
              'Stat Type': stat.get('type', 'N/A'),
              'Total': stat.get('value', 0)
          }
          team_stats_data.append(stat_info)

  # Create a DataFrame and converts "NAN" to 0 value
  df_team_stats = pd.DataFrame(team_stats_data)
  df_team_stats  = df_team_stats.pivot(index=['Team ID', 'Competition ID', 'Competition Name','Match ID', 'Week', 'Stage ID', 'Stage Name'], columns='Stat Type', values='Total').reset_index()
  df_team_stats = df_team_stats.fillna(0)

  # Convert all stat columns (excluding 'Team ID' and 'Match ID') to floats
  stat_cols = [col for col in df_team_stats.columns if col not in ['Team ID', 'Match ID', 'Competition ID', 'Competition Name','Week', 'Stage ID', 'Stage Name']]
  for col in stat_cols:
    df_team_stats[col] = pd.to_numeric(df_team_stats[col], errors='coerce').fillna(0).astype(float)

  return df_team_stats

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_matchTeamStats_data(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_teamStats_df = pd.concat(all_data, ignore_index=True)
    print(final_teamStats_df)
else:
    print("No match data found.")

# ====================================
# STEP 4: GET MA12 (Player xG Stats) - Player xG Stats fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA12
# ====================================

def process_xG_matchPlayerStats_data(match_id):

  # API Parameters
  requestParameters = {
      "_fmt" : "json",
      "fx" : match_id,
      "_rt" : "b"
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchexpectedgoals/{0}/?'.format(outletApiKey)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)

  # Create a DataFrame for xG player stats
  player_xGstats_data = []

  # Check response Status
  if response.status_code == 200:
    data = response.json()

    # Extract data
    match_info = data.get('matchInfo', {})
    competition_info = match_info.get('competition', {})
    stage_info = match_info.get('stage', {})
    live_data = data.get('liveData', {})
    line_ups = live_data.get('lineUp', [])


    # Extract player stats
    for line_up in line_ups:
      for player in line_up.get('player', []):
        player_entry = {
            'Match ID':match_info.get('id','N/A'),
            'Competition ID': competition_info.get('id', 'N/A'),
            'Competition Name': competition_info.get('name', 'N/A'),
            'Week': match_info.get('week', 'N/A'),
            'Stage ID': stage_info.get('id', 'N/A'),
            'Stage Name': stage_info.get('name', 'N/A'),
            'Player ID': player.get('playerId', 'N/A'),
            'First Name': player.get('firstName', 'N/A'),
            'Last Name': player.get('lastName', 'N/A'),
            'Match Name': player.get('matchName', 'N/A'),
            'Shirt Number': player.get('shirtNumber', 'N/A'),
            'Position': player.get('position', 'N/A'),
            'Position Side': player.get('positionSide', 'N/A'),
            'Formation Place': player.get('formationPlace', 'N/A'),
        }

        for stat in player.get('stat', []):
          stat_type = stat.get('type', '')
          stat_value = stat.get('value', 0)
          player_entry[stat_type] = stat_value

        player_xGstats_data.append(player_entry)

  else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)  # Print the API response for debugging
    return None # Return None if the API call failed

  # Create a DataFrame and converts "NAN" to 0 value
  df_player_xGstats = pd.DataFrame(player_xGstats_data).fillna(0)

  # Convert all stat columns (excluding metadata) to numeric
  non_stat_cols = ['Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage Name', 'Stage ID','Player ID', 'First Name', 'Last Name', 'Match Name',
                 'Shirt Number', 'Position', 'Position Side', 'Formation Place']
  stat_cols = [col for col in df_player_xGstats.columns if col not in non_stat_cols]

  for col in stat_cols:
    df_player_xGstats[col] = pd.to_numeric(df_player_xGstats[col], errors='coerce').fillna(0)

  # Optionally, convert all floats to ints if appropriate
  df_player_xGstats[stat_cols] = df_player_xGstats[stat_cols].astype(float)
  return df_player_xGstats

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_xG_matchPlayerStats_data(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_xG_playerStats_df = pd.concat(all_data, ignore_index=True)
    print(final_xG_playerStats_df)
else:
    print("No match data found.")

# ====================================
# STEP 5: GET MA12 (Player xG Stats) - Player xG Stats fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA12
# ====================================

def process_xG_matchTeamStats_data(match_id):

  # API Parameters
  requestParameters = {
      "_fmt" : "json",
      "fx" : match_id,
      "_rt" : "b"
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchexpectedgoals/{0}/?'.format(outletApiKey)
  response = requests.get(
      sdapi_get_url,
      headers=requestHeaders(),
      params=requestParameters)

  # Check response Status
  if response.status_code == 200:
    data = response.json()
  else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)  # Print the API response for debugging

  # Extract data
  match_info = data.get('matchInfo', {})
  competition_info = match_info.get('competition', {})
  stage_info = match_info.get('stage', {})
  live_data = data.get('liveData', {})
  line_ups = live_data.get('lineUp', [])

  # Create a DataFrame for xG team stats
  team_xGstats_data = []

  # Extract team stats
  for team_stats in line_ups:
      team_id = team_stats['contestantId']
      for stat in team_stats['stat']:
          stat_info = {
              'Match ID':match_info.get('id','N/A'),
              'Competition ID': competition_info.get('id', 'N/A'),
              'Competition Name': competition_info.get('name', 'N/A'),
              'Week': match_info.get('week', 'N/A'),
              'Stage ID': stage_info.get('id', 'N/A'),
              'Stage Name': stage_info.get('name', 'N/A'),
              'Team ID': team_id,
              'Stat Type': stat.get('type', 'N/A'),
              'Total': stat.get('value', 0)
          }
          team_xGstats_data.append(stat_info)

  # Create a DataFrame and converts "NAN" to 0 value
  df_xGteam_stats = pd.DataFrame(team_xGstats_data)
  df_xGteam_stats  = df_xGteam_stats.pivot(index=['Team ID', 'Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name'], columns='Stat Type', values='Total').reset_index()
  df_xGteam_stats = df_xGteam_stats.fillna(0)

  # Convert all stat columns (excluding 'Team ID' and 'Match ID') to floats
  stat_cols = [col for col in df_xGteam_stats.columns if col not in ['Team ID', 'Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name']]
  for col in stat_cols:
    df_xGteam_stats[col] = pd.to_numeric(df_xGteam_stats[col], errors='coerce').fillna(0).astype(float)

  return df_xGteam_stats

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_xG_matchTeamStats_data(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_xG_teamStats_df = pd.concat(all_data, ignore_index=True)
    print(final_xG_teamStats_df)
else:
    print("No match data found.")

# ====================================
# STEP 6: GET MA12 (Player xG Events) - Player xG Events fethcer and DF storage
# Function to iterate over matchId_df and make GET request to MA12
# ====================================

def process_xG_match_events(match_id):

  # API Parameters
  requestParameters = {
      "_rt" : "b",
      "_fmt" : "json",
      "fx" : match_id
      }

  # GET API
  sdapi_get_url = 'https://api.performfeeds.com/soccerdata/matchexpectedgoals/{0}/?'.format(outletApiKey)

  response = requests.get(
      sdapi_get_url,
      headers = requestHeaders(),
      params=requestParameters
      )

  # Check response Status
  if response.status_code == 200:
      data = response.json()
  else:
      print(f"Error: API request failed with status code {response.status_code}")
      print(response.text)  # Print the API response for debugging

  # Extract event data
  match_info = data.get('matchInfo', {})
  competition_info = match_info.get('competition', {})
  stage_info = match_info.get('stage', {})
  live_data = data.get('liveData', {})
  xG_events = live_data.get('event', [])


  # Find all unique qualifier IDs
  qualifier_ids = {'321','322'}

  # Initialize DataFrame columns
  columns = [
      'Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name','EventId', 'timeStamp', 'contestantId', 'periodId', 'timeMin', 'timeSec',
      'playerId', 'playerName', 'typeId', 'outcome', 'x', 'y'] + [f'qualifier {qid}' for qid in qualifier_ids]

  # Create a DataFrame for xG events
  xG_events_data = []

  for event in xG_events:
    xG_event_info = {
        'Match ID':match_info.get('id','N/A'),
        'Competition ID': competition_info.get('id', 'N/A'),
        'Competition Name': competition_info.get('name', 'N/A'),
        'Week': match_info.get('week', 'N/A'),
        'Stage ID': stage_info.get('id', 'N/A'),
        'Stage Name': stage_info.get('name', 'N/A'),
        'EventId': event.get('eventId', None),
        'typeId': event.get('typeId', None),
        'periodId': event.get('periodId', None),
        'timeMin': event.get('timeMin', None),
        'timeSec': event.get('timeSec', None),
        'contestantId': event.get('contestantId', None),
        'playerId': event.get('playerId', None),
        'playerName': event.get('playerName', None),
        'outcome': event.get('outcome', None),
        'x': event.get('x', None),
        'y': event.get('y', None),
        'timeStamp': event.get('timeStamp', None),
        }

    for qid in qualifier_ids:
      xG_event_info[f'qualifier {qid}'] = 0

    # Update with actual qualifier values
    for q in event.get('qualifier', []):
      xG_event_info[f'qualifier {q["qualifierId"]}'] = q.get('value', None)

    xG_events_data.append(xG_event_info)

  xG_events_df = pd.DataFrame(xG_events_data, columns=columns)
  return xG_events_df

# Get the match IDs
matchId_df = get_match_ids()

# Initialize an empty list to store DataFrames
all_data = []

# Iterate over each match ID and process the data
for match_id in matchId_df['Match ID']:
    df = process_xG_match_events(match_id)
    if df is not None:
        all_data.append(df)
    time.sleep(delay_seconds) # Add delay here

# Combine all DataFrames if there is data
if all_data:
    final_xG_events_df = pd.concat(all_data, ignore_index=True)
    print(final_xG_teamStats_df)
else:
    print("No match data found.")

# ==============================
# EXCEL EXPORT
# ==============================

def export_to_excel(df, filename):
    if df.empty:
        print(f"DataFrame '{filename}' is empty. Skipping export.")
        return

    df.to_excel(filename, index=False)
    print(f"Data has been saved to '{filename}'")

    try:
        files.download(filename)
    except ImportError:
        print("(files.download skipped")


# ==============================
# Full Pipeline Main
# ==============================

def main():
# ============================================================
    # STEP 1: GET MA1 - Match IDs for CHIVAS games
# ============================================================

    matchId_df = get_match_ids()
    display(matchId_df)

    if matchId_df.empty:
        print("No Match IDs found.")
        return

# ============================================================
    # STEP 2: GET MA2 - Player Stats
# ============================================================

    # Create a DataFrame for xG player stats
    player_stats_data = []

    # Iterate over each match ID and process the data
    for match_id in matchId_df['Match ID']:
        df = process_matchPlayerStats_data(match_id)
        if not df.empty:
            player_stats_data.append(df)
        time.sleep(delay_seconds) # Add delay here

    # Combine all DataFrames if there is data
    if player_stats_data:
        final_playerStats_df = pd.concat(player_stats_data, ignore_index=True)
        display(final_playerStats_df)
        export_to_excel(final_playerStats_df, "Aggregated_player_stats.xlsx")
    else:
        print("No Player Stats found.")

# ============================================================
    # STEP 3: GET MA2 - Team Stats
# ============================================================

    # Create a DataFrame for team stats
    team_stats_data = []

    # Iterate over each match ID and process the data
    for match_id in matchId_df['Match ID']:
        df = process_matchTeamStats_data(match_id)
        if not df.empty:
            team_stats_data.append(df)
        time.sleep(delay_seconds) # Add delay here

    # Combine all DataFrames if there is data
    if team_stats_data:
        final_teamStats_df = pd.concat(team_stats_data, ignore_index=True)
        display(final_teamStats_df)
        export_to_excel(final_teamStats_df, "Aggregated_team_stats.xlsx")
    else:
        print("No Team Stats found.")

# ============================================================
    # STEP 4: GET MA12 - Player xG Stats
# ============================================================

    # Create a DataFrame for xG player stats
    player_xGstats_data = []

    # Iterate over each match ID and process the data
    for match_id in matchId_df['Match ID']:
        df = process_xG_matchPlayerStats_data(match_id)
        if not df.empty:
            player_xGstats_data.append(df)
        time.sleep(delay_seconds) # Add delay here

    # Combine all DataFrames if there is data
    if player_xGstats_data:
        final_xG_playerStats_df = pd.concat(player_xGstats_data, ignore_index=True)
        display(final_xG_playerStats_df)
        export_to_excel(final_xG_playerStats_df, "Aggregated xG Player Stats.xlsx")
    else:
        print("No Team Stats found.")

# ============================================================
    # STEP 5: GET MA12 - Team xG Stats
# ============================================================

    # Create a DataFrame for xG team stats
    team_xGstats_data = []

    # Iterate over each match ID and process the data
    for match_id in matchId_df['Match ID']:
        df = process_matchTeamStats_data(match_id)
        if not df.empty:
            team_xGstats_data.append(df)
        time.sleep(delay_seconds) # Add delay here

    # Combine all DataFrames if there is data
    if team_xGstats_data:
        final_xG_teamStats_df = pd.concat(team_xGstats_data, ignore_index=True)
        display(final_xG_teamStats_df)
        export_to_excel(final_xG_teamStats_df, "Aggregated xG Team Stats.xlsx")
    else:
        print("No Team Stats found.")

# ============================================================
    # STEP 6: GET MA12 - xG Match Events
# ============================================================

    # Create a DataFrame for xG events
    xG_events_data = []

    # Iterate over each match ID and process the data
    for match_id in matchId_df['Match ID']:
        df = process_xG_match_events(match_id)
        if not df.empty:
            xG_events_data.append(df)
        time.sleep(delay_seconds) # Add delay here

    # Combine all DataFrames if there is data
    if xG_events_data:
        final_xG_events_df = pd.concat(xG_events_data, ignore_index=True)
        display(final_xG_events_df)
        export_to_excel(final_xG_events_df, "Aggregated xG events.xlsx")
    else:
        print("No Team Stats found.")

main()